In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy

# CHSH-Ungleichung

*Nutzungsschätzung: Zwei Minuten auf einem Heron r2-Prozessor (HINWEIS: Dies ist nur eine Schätzung. Deine Laufzeit kann variieren.)*
## Hintergrund
In diesem Tutorial führst du ein Experiment auf einem Quantencomputer aus, um die Verletzung der CHSH-Ungleichung mit dem Estimator-Primitiv zu demonstrieren.

Die CHSH-Ungleichung, benannt nach den Autoren Clauser, Horne, Shimony und Holt, wird verwendet, um Bells Theorem (1969) experimentell zu beweisen. Dieses Theorem besagt, dass lokale Theorien mit verborgenen Variablen einige Konsequenzen der Verschränkung in der Quantenmechanik nicht erklären können. Die Verletzung der CHSH-Ungleichung wird verwendet, um zu zeigen, dass die Quantenmechanik mit lokalen Theorien mit verborgenen Variablen nicht vereinbar ist. Dies ist ein wichtiges Experiment zum Verständnis der Grundlagen der Quantenmechanik.

Der Nobelpreis für Physik 2022 wurde an Alain Aspect, John Clauser und Anton Zeilinger verliehen, unter anderem für ihre Pionierarbeit in der Quanteninformationswissenschaft und insbesondere für ihre Experimente mit verschränkten Photonen, die die Verletzung der Bellschen Ungleichungen demonstrierten.
## Anforderungen
Stelle vor Beginn dieses Tutorials sicher, dass du Folgendes installiert hast:

* Qiskit SDK v1.0 oder neuer, mit [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)-Unterstützung
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.22 oder neuer
## Setup

In [1]:
# General
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Plotting routines
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

## Schritt 1: Klassische Eingaben auf ein Quantenproblem abbilden
Für dieses Experiment erstellen wir ein verschränktes Paar, bei dem wir jedes Qubit auf zwei verschiedenen Basen messen. Wir bezeichnen die Basen für das erste Qubit mit $A$ und $a$ und die Basen für das zweite Qubit mit $B$ und $b$. Dies ermöglicht es uns, die CHSH-Größe $S_1$ zu berechnen:

$$
S_1 = A(B-b) + a(B+b).
$$

Jede Observable ist entweder $+1$ oder $-1$. Klar ist, dass einer der Terme $B\pm b$ gleich $0$ sein muss und der andere $\pm 2$ sein muss. Daher ist $S_1 = \pm 2$. Der Durchschnittswert von $S_1$ muss die Ungleichung erfüllen:

$$
|\langle S_1 \rangle|\leq 2.
$$

Die Entwicklung von $S_1$ in Bezug auf $A$, $a$, $B$ und $b$ ergibt:

$$
|\langle S_1 \rangle| = |\langle AB \rangle - \langle Ab \rangle + \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Du kannst eine weitere CHSH-Größe $S_2$ definieren:

$$
S_2 = A(B+b) - a(B-b),
$$

Dies führt zu einer weiteren Ungleichung:

$$
|\langle S_2 \rangle| = |\langle AB \rangle + \langle Ab \rangle - \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Wenn die Quantenmechanik durch lokale Theorien mit verborgenen Variablen beschrieben werden kann, müssen die vorherigen Ungleichungen wahr sein. Wie jedoch in diesem Tutorial demonstriert wird, können diese Ungleichungen auf einem Quantencomputer verletzt werden. Daher ist die Quantenmechanik nicht mit lokalen Theorien mit verborgenen Variablen vereinbar.
Falls du mehr Theorie lernen möchtest, erkunde [Entanglement in Action](/learning/courses/basics-of-quantum-information/entanglement-in-action/chsh-game) mit John Watrous.
Du wirst ein verschränktes Paar zwischen zwei Qubits in einem Quantencomputer erstellen, indem du den Bell-Zustand $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$ erzeugst. Mit dem Estimator-Primitiv kannst du direkt die benötigten Erwartungswerte ($\langle AB \rangle, \langle Ab \rangle, \langle aB \rangle$ und $\langle ab \rangle$) erhalten, um die Erwartungswerte der beiden CHSH-Größen $\langle S_1\rangle$ und $\langle S_2\rangle$ zu berechnen. Vor der Einführung des Estimator-Primitivs musstest du die Erwartungswerte aus den Messergebnissen konstruieren.

Du wirst das zweite Qubit in den $Z$- und $X$-Basen messen. Das erste Qubit wird ebenfalls in orthogonalen Basen gemessen, jedoch mit einem Winkel bezüglich des zweiten Qubits, den wir zwischen $0$ und $2\pi$ variieren werden. Wie du sehen wirst, macht das Estimator-Primitiv das Ausführen parametrisierter Schaltkreise sehr einfach. Anstatt eine Reihe von CHSH-Schaltkreisen zu erstellen, musst du nur *einen* CHSH-Schaltkreis mit einem Parameter erstellen, der den Messwinkel angibt, und eine Reihe von Phasenwerten für den Parameter.

Schließlich wirst du die Ergebnisse analysieren und gegen den Messwinkel auftragen. Du wirst sehen, dass für einen bestimmten Bereich von Messwinkeln die Erwartungswerte der CHSH-Größen $|\langle S_1\rangle| > 2$ oder $|\langle S_2\rangle| > 2$ sind, was die Verletzung der CHSH-Ungleichung demonstriert.

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_kingston'

### Create a parameterized CHSH circuit

First, we write the circuit with the parameter $\theta$, which we call `theta`. The [`Estimator` primitive](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) can enormously simplify circuit building and output analysis by directly providing expectation values of observables. Many problems of interest, especially for near-term applications on noisy systems, can be formulated in terms of expectation values. `Estimator` (V2) primitive can automatically change measurement basis based on the supplied observable.

In [3]:
theta = Parameter("$\\theta$")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)
chsh_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif" alt="Output of the previous code cell" />

### Einen parametrisierten CHSH-Schaltkreis erstellen
Zuerst schreiben wir den Schaltkreis mit dem Parameter $\theta$, den wir `theta` nennen. Das [`Estimator`-Primitiv](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) kann den Schaltkreisaufbau und die Ausgabeanalyse enorm vereinfachen, indem es direkt Erwartungswerte von Observablen liefert. Viele interessante Probleme, insbesondere für kurzfristige Anwendungen auf verrauschten Systemen, können in Form von Erwartungswerten formuliert werden. Das `Estimator` (V2)-Primitiv kann automatisch die Messbasis basierend auf der bereitgestellten Observable ändern.

In [4]:
number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
# Phases need to be expressed as list of lists in order to work
individual_phases = [[ph] for ph in phases]

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif)

### Eine Liste von Phasenwerten erstellen, die später zugewiesen werden
Nachdem du den parametrisierten CHSH-Schaltkreis erstellt hast, erstelle eine Liste von Phasenwerten, die dem Schaltkreis im nächsten Schritt zugewiesen werden. Du kannst den folgenden Code verwenden, um eine Liste von 21 Phasenwerten im Bereich von $0$ bis $2 \pi$ mit gleichem Abstand zu erstellen, d.h. $0$, $0.1 \pi$, $0.2 \pi$, ..., $1.9 \pi$, $2 \pi$.

In [5]:
# <CHSH1> = <AB> - <Ab> + <aB> + <ab> -> <ZZ> - <ZX> + <XZ> + <XX>
observable1 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", -1), ("XZ", 1), ("XX", 1)]
)

# <CHSH2> = <AB> + <Ab> - <aB> + <ab> -> <ZZ> + <ZX> - <XZ> + <XX>
observable2 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", 1), ("XZ", -1), ("XX", 1)]
)

### Observablen
Nun benötigen wir Observablen, aus denen wir die Erwartungswerte berechnen können. In unserem Fall betrachten wir orthogonale Basen für jedes Qubit, wobei die parametrisierte $Y$-Rotation für das erste Qubit die Messbasis nahezu kontinuierlich in Bezug auf die Basis des zweiten Qubits variiert. Wir wählen daher die Observablen $ZZ$, $ZX$, $XZ$ und $XX$.

In [6]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

chsh_isa_circuit = pm.run(chsh_circuit)
chsh_isa_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif" alt="Output of the previous code cell" />

## Schritt 2: Problem für die Ausführung auf Quantenhardware optimieren

Um die Gesamtausführungszeit des Jobs zu reduzieren, akzeptieren V2-Primitive nur Schaltkreise und Observablen, die den vom Zielsystem unterstützten Anweisungen und der Konnektivität entsprechen (bezeichnet als Instruction Set Architecture (ISA)-Schaltkreise und -Observablen).

### ISA-Schaltkreis

In [7]:
isa_observable1 = observable1.apply_layout(layout=chsh_isa_circuit.layout)
isa_observable2 = observable2.apply_layout(layout=chsh_isa_circuit.layout)

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif)

### ISA-Observablen

Ebenso müssen wir die Observablen transformieren, um sie backend-kompatibel zu machen, bevor wir Jobs mit [`Runtime Estimator V2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2#run) ausführen. Wir können die Transformation mit der `apply_layout`-Methode des `SparsePauliOp`-Objekts durchführen.

In [8]:
# To run on a local simulator:
# Use the StatevectorEstimator from qiskit.primitives instead.

estimator = Estimator(mode=backend)

pub = (
    chsh_isa_circuit,  # ISA circuit
    [[isa_observable1], [isa_observable2]],  # ISA Observables
    individual_phases,  # Parameter values
)

job_result = estimator.run(pubs=[pub]).result()

## Schritt 3: Ausführen mit Qiskit-Primitiven
Um das gesamte Experiment in einem einzigen Aufruf des [`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) auszuführen.
Wir können ein [Qiskit Runtime `Estimator`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2)-Primitiv erstellen, um unsere Erwartungswerte zu berechnen. Die `EstimatorV2.run()`-Methode akzeptiert ein Iterable von `primitive unified blocs (PUBs)`. Jedes PUB ist ein Iterable im Format `(circuit, observables, parameter_values: Optional, precision: Optional)`.

In [9]:
chsh1_est = job_result[0].data.evs[0]
chsh2_est = job_result[0].data.evs[1]

In [10]:
fig, ax = plt.subplots(figsize=(10, 6))

# results from hardware
ax.plot(phases / np.pi, chsh1_est, "o-", label="CHSH1", zorder=3)
ax.plot(phases / np.pi, chsh2_est, "o-", label="CHSH2", zorder=3)

# classical bound +-2
ax.axhline(y=2, color="0.9", linestyle="--")
ax.axhline(y=-2, color="0.9", linestyle="--")

# quantum bound, +-2√2
ax.axhline(y=np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.axhline(y=-np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.fill_between(phases / np.pi, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7)
ax.fill_between(phases / np.pi, -2, -2 * np.sqrt(2), color="0.6", alpha=0.7)

# set x tick labels to the unit of pi
ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

# set labels, and legend
plt.xlabel("Theta")
plt.ylabel("CHSH witness")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/f6267448-0.avif" alt="Output of the previous code cell" />

In the figure, the lines and gray areas delimit the bounds; the outer-most (dash-dotted) lines delimit the quantum-bounds ($\pm 2$), whereas the inner (dashed) lines delimit the classical bounds ($\pm 2\sqrt{2}$). You can see that there are regions where the CHSH witness quantities exceeds the classical bounds. Congratulations! You have successfully demonstrated the violation of CHSH inequality in a real quantum system!

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3xxAgm1SF1wGp9k)